In [ ]:
import re
from importlib.metadata import version
import tiktoken
print ("tiktoken version:", version("tiktoken"))
import torch
from torch.utils.data import Dataset, DataLoader


###################################2章 テキストデータの準備###############################################

#『the-verdict』という小説をベースにLLMを構築します。
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()



#スライディングウィンドウ形式、バッチ入力と目的変数のためのデータセット
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)#txtを数値トークン化する

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]#「the-verdict」トークンをmax_lengthの長さのシーケンスに分割し、input_chunkに代入
            target_chunk = token_ids[i+1:i + max_length + 1]#「the-verdict」トークンをmax_lengthの長さのシーケンスに分割し、target_chunkに代入
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]



#入力変数と目的変数のペアでバッチを生成するデータローダーの作成
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                           stride=128, shuffle=True, drop_last=True, num_workers=0):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    
    return dataloader



#インスタンス化する
max_length = 4 #任意の値
dataloader = create_dataloader_v1(
    raw_text,
    drop_last = True,
    batch_size=8,#任意の値
    max_length=max_length,
    stride=max_length,
    shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)


#埋め込み(最初のbatch_size個のtokenを埋め込んでいる)
vocab_size = 50257 #50257==tokenizer.n_vocab
output_dim = 256 
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim) 
#↑vocab_sizeの数値トークンを256次元で表現する
token_embeddings = token_embedding_layer(inputs) 
#↑batch_size×max_lengthテンソルのInputsを256次元で表現する



#絶対位置埋め込み(max_length内で位置を定め、埋め込んでいます)
output_dim = 256 
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)#256次元ベクトルへの埋め込み
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
input_embeddings = token_embeddings + pos_embeddings



###################################2章終わり###############################################
  